# Assignment 1 IR | Boolean Model

## 1. Libraries Initialization

In [1]:
import sys
import string
from nltk.stem import PorterStemmer
import tkinter as tk


## 2. Reading StopWords

In [2]:
def readStopWords():
    f = open("stopwords.txt", "r")
    stopWords = f.read().split()
  
    # making a Frozen Set as we want to use the "Hashing" part of it when we are processing files to ignore stopWords
    stepWordsStem = set()
    # We are stemming Stop words too to be able to remove all stems of a stopWord 
    # i.e if have is a stopword we want to ignore have, having, haven't and so on all stems of have 
    
    for stopword in stopWords:
        stepWordsStem.add(ps.stem(stopword))
    stepWordsStem = frozenset(stepWordsStem)
    f.close()
    print(stopWords)
    return stepWordsStem

## 3. Finding Words from Stories

In [3]:
def readFile(file):
#     f = open(f"./ShortStories/{file}.txt", encoding="utf8")
    f=open('./ShortStories/{}.txt'.format(file), encoding='utf-8')
    
    #   Read and Lower Case 
    data = f.read().lower()    
        
    #   Removing Punctuations
    data = data.translate(str.maketrans('','',string.punctuation))

    # We have identified all unique characters and numbers and removed all others but alphabets       
    # '9', '0', '—', 'i', 'u', 'q', '2', 'e', 'ª', 'z', 'y', '”', '8', 'd', 'v', 'o', 'm', 'f', 'b', '¨', '7', '1', 'a', '’', 'l', 'w', '§', 'j', '“', 'x', 'g', '©', 'p', '6', '3', 'r', 'c', 't', 'ã', '5', '´', '¯', '‘', 'h', '4', 's', 'k', 'n
    data = data.translate(str.maketrans('','','1234567890©§“”‘’"´¨¯—ªã'))

    # Word to word checking of file and adding to Dictionary
    
#     for word in data.split():
        
#         word = ps.stem(word)
#         if word not in stopWords:
#             print(word, ps.stem(word))

#             if word in dictionary.keys():
#                 dictionary[word].add(file)
#             else:
#                 dictionary[word] = {file}
    
    for i, word in enumerate(data.split()):
        word = ps.stem(word)
        if word not in stopWords:

            if word in dictionary.keys():
                if file in dictionary[word].keys():
                    dictionary[word][file].add(i)
                else:
                    dictionary[word][file] = {i}
            else:
                dictionary[word] = {file: {i}}

    f.close()
    return

dictionary = {}
ps = PorterStemmer()
stopWords = readStopWords()
# Run for Each File
for x in range(1,51):
    readFile(x)
    
    
print("Dictionary Length: ", len(dictionary))
# print(dictionary)

['a', 'is', 'the', 'of', 'all', 'and', 'to', 'can', 'be', 'as', 'once', 'for', 'at', 'am', 'are', 'has', 'have', 'had', 'up', 'his', 'her', 'in', 'on', 'no', 'we', 'do']
Dictionary Length:  6763


## 4. Writing Dictionary

In [4]:
f = open("dictionary.txt", "w")

with open('dictionary.txt', 'w') as the_file:
    for word in dictionary:
        f.write('{}\n'.format(word))
    f.close()

## 5. Inverted Index

In [5]:
f = open("invertedIndex.txt", "w")

with open('invertedIndex.txt', 'w') as the_file:
    f.write('{')
    for word in dictionary:
        f.write('"{}": {},\n'.format(word, list(dictionary[word].keys())))
#         print(list(dictionary[word].keys()))
    f.write('}')
    f.close()

## 5. Positional Index

In [6]:
f = open("positionalIndex.txt", "w")
    
with open('positionalIndex.txt', 'w') as the_file:
    f.write('{')
    for word in dictionary:
#         print('"{}":"{}"\n'.format(word, dictionary[word] ))
        f.write("'{}':{},\n".format(word, dictionary[word] ))
    f.write('}')

    f.close()
                       

## 6. Reading Inverted Index

In [7]:
import ast
f = open("invertedIndex.txt", "r")
data = f.read()
invertedIndex = ast.literal_eval(data)
print(invertedIndex['beard'])

[1, 2, 4, 6, 11, 20, 21, 23, 25, 26, 31, 34, 40, 44]


## 7. Reading Positional Index

In [8]:
import ast
f = open("positionalIndex.txt", "r")
data = f.read()
positionalIndex = ast.literal_eval(data)
print(positionalIndex['beard'])

{1: {737, 1978}, 2: {2074, 4442, 591}, 4: {752}, 6: {152}, 11: {205, 719}, 20: {36, 511}, 21: {134}, 23: {2145, 2882, 3641, 4382, 1151}, 25: {5758, 559}, 26: {972}, 31: {12}, 34: {490, 822}, 40: {674}, 44: {909}}


## 8. Normal Query

In [9]:
def normalQuery(query):

    currentIndex = 0
    resultDocs = set()
    universalSet = set(range(1,51))
    error = False
#     For First Word
    if query[currentIndex] == "not":
        resultForWord = set(invertedIndex[ps.stem(query[currentIndex + 1])]) 
#       taking Not
        resultDocs = resultDocs.union(universalSet - resultForWord)
        currentIndex += 2
    else:
        resultDocs = resultDocs.union(set(invertedIndex[ps.stem(query[currentIndex])]))
        currentIndex += 1
              
#   For Second Word

    if len(query) > currentIndex:
    
#     Does the second word have AND or OR
        if query[currentIndex] == "and":
        
#           Is the second word with a not ?
            if query[currentIndex + 1] == "not":
                resultForWord = set(invertedIndex[ps.stem(query[currentIndex + 2])])      
#               taking Not
                takingNot = universalSet - resultForWord
#               Intersection Due to 'AND'
                resultDocs = resultDocs.intersection(takingNot)
                currentIndex += 3
            else:
                resultDocs = resultDocs.intersection(set(invertedIndex[ps.stem(query[currentIndex + 1])]))
                currentIndex += 2

                
        elif query[currentIndex] == "or":
            if query[currentIndex] == "not":
                resultForWord = set(invertedIndex[ps.stem(query[currentIndex + 2])])
                
#               ----------------taking Not
                resultDocs = resultDocs.union(universalSet - resultForWord)

                currentIndex += 3
            else:
                resultDocs = resultDocs.union(set(invertedIndex[ps.stem(query[currentIndex + 1])]))
                currentIndex += 2
        else: 
            print("Invalid Query, kindly mention any of (AND, OR, NOT) operators between each word")
            error = True;
            
#   For Third Word

    if len(query) > currentIndex and not error:
        
#     Does the third word have AND or OR
        if query[currentIndex] == "and":
        
#           Is the third word with a not ?
            if query[currentIndex + 1] == "not":
                resultForWord = set(invertedIndex[ps.stem(query[currentIndex + 2])])      
#               taking Not
                takingNot = universalSet - resultForWord
    
#               Intersection Due to 'AND'
                resultDocs = resultDocs.intersection(takingNot)
                currentIndex += 3
            else:
#               union due to OR
                resultDocs = resultDocs.intersection(set(invertedIndex[ps.stem(query[currentIndex + 1])]))
                currentIndex += 2

                
        elif query[currentIndex] == "or":
            if query[currentIndex] == "not":
                resultForWord = set(invertedIndex[ps.stem(query[currentIndex + 2])])      
#               taking Not
                resultDocs = resultDocs.union(universalSet - resultForWord)

                currentIndex += 3
            else:
#               union due to OR
                resultDocs = resultDocs.union(set(invertedIndex[ps.stem(query[currentIndex + 1])]))
                currentIndex += 2
        else: 
            print("Invalid Query, kindly mention any of (AND, OR, NOT) operators between each word")
            error = True
    
    if(not error):
        print(resultDocs)
        return resultDocs
        
        
        

## 9. Proximity Query

In [10]:
def proximityQuery(word1, word2, k):
    k = int(k)
    qualified = set()
    firstWordFiles = positionalIndex[ps.stem(word1)]
    secondWordFiles = positionalIndex[ps.stem(word2)]
    
#     Finding Sets of each dictionary keys
    word1Set = set(firstWordFiles)
    word2Set = set(secondWordFiles)
    
#     Finding common keys
    result = word1Set.intersection(word2Set)

#     Finding compliment of keys with their respective sets to find uncommon keys 
    word1Set = word1Set - result
    word2Set = word2Set - result
    
#     Removing the uncommon keys from word Files
    for uslessfile in word1Set:
        firstWordFiles.pop(uslessfile)

    for uslessfile in word2Set:
        secondWordFiles.pop(uslessfile)
    
#     Finding K distance files
    for file in secondWordFiles:
        for locationOfWord in secondWordFiles[file]:
            result = set(range(locationOfWord - k - 1,locationOfWord + k + 1)).intersection(firstWordFiles[file])
            if(len(result)):
                qualified.add(file)

    print(qualified)
    return qualified

proximityQuery('smiling', 'face', 3)

        
        
        

{25, 11, 6, 22}


{6, 11, 22, 25}

## 9. Handling a Query

In [18]:
# query = "filling room /1".lower().split()
import sys
# print(query)
def handle_query():
#     query = inputBox.get("1.0","end-1c")
    query = input()
    query = query.lower().split()
    if query[len(query) - 1][0] == '/':
        # It is a proximity Query
        result = proximityQuery(query[0], query[1], query[2][1])
#         result_box.delete(0.0, 'end')
#         result_box.insert('end', result)
        print(result)
    else:
        # It is a normal Query
        result = normalQuery(query)
#         result_box.delete(0.0, 'end')
#         result_box.insert('end', result)
        print(result)
    


In [23]:
handle_query()

lover bullet /3
{15}
{15}


## 10. Main Function GUI

In [14]:
root = tk.Tk()
root.geometry("600x400")
root.title('Boolean Retreival Model')


tk.Label(root, text="Boolean Retreival Model", font=("Roboto", 20)).pack(pady=(5, 10))


tk.Label(root, text="Search...", font=("Roboto", 12)).pack(pady=(10, 2))


inputBox = tk.Entry(root, font=("Roboto", 12), width=45)

inputBox.pack()


submitButton = tk.Button(root, text="SUBMIT", width=40, command=handle_query).pack()


tk.Label(root, text="Result Documents", font=("Roboto", 12)).pack(pady=(10, 2))

root.bind('<Return>', handle_query)

    # resultBox
result_box = tk.Text(root, width=50, height=5, font=("Roboto", 12), wrap='word')

result_box.pack(pady=(0, 10))


root.mainloop()
    
    

Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/maxplanko89/anaconda3/lib/python3.8/tkinter/__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-13-b8f82868c207>", line 5, in handle_query
    query = inputBox.get("1.0","end-1c")
TypeError: get() takes 1 positional argument but 3 were given


# Modules

## Finding All Characters used before translating

In [26]:
allchars = set()    
for word in dictionary:
        for letter in word:
            allchars.add(letter)
print(allchars)

{'9', '0', '—', 'i', 'u', 'q', '2', 'e', 'ª', 'z', 'y', '”', '8', 'd', 'v', 'o', 'm', 'f', 'b', '¨', '7', '1', 'a', '’', 'l', 'w', '§', 'j', '“', 'x', 'g', '©', 'p', '6', '3', 'r', 'c', 't', 'ã', '5', '´', '¯', '‘', 'h', '4', 's', 'k', 'n'}


## String Processing 

In [35]:
import sys
import string 


    #   Read a Line and work on it and repeat
line = "my long string's big 9 sentance anti-social different 1str@ing!"
        
    #   End of line Found

line = line.lower()
line = line.translate(str.maketrans('','',string.punctuation))
line = line.translate(str.maketrans('','','1234567890'))
line = line.translate(str.maketrans('','','“”‘’"—'))

# words = word_tokenize(line)

print("Line: ", line)
# print("Words: ", words)

Line:  my long strings big  sentance antisocial my different string


## Stemming

In [13]:
# import nltk
# nltk.download()
# importing modules
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
words = word_tokenize("my long string's big sentance that-make different 1str@ing!")
ps = PorterStemmer()

for w in words:
    print(w, " \t\t:\t", ps.stem(w))

my  		:	 my
long  		:	 long
string  		:	 string
's  		:	 's
big  		:	 big
sentance  		:	 sentanc
that-make  		:	 that-mak
different  		:	 differ
1str  		:	 1str
@  		:	 @
ing  		:	 ing
!  		:	 !


## StopWords

In [14]:
import sys
import string
from nltk.stem import PorterStemmer

f = open("stopwords.txt", "r")
stopwords = []
while(1):
    line = f.readline().strip()
    if(not line):
        break
    stopwords.append(line)
f.close()
for psword in stopwords:
    print(psword, ps.stem(psword))

a a
is is
the the
of of
all all
and and
to to
can can
be be
as as
once onc
for for
at at
am am
are are
has ha
have have
had had
up up
his hi
her her
in in
on on
no no
we we
do do


## Removing StopWords

In [ ]:
f = open("stopwords.txt", "r")
stopWords = f.read().split()
for stopword in stopWords:
    dictionary.pop(ps.stem(stopword))
f.close()
print("Dictionary Length: ", len(dictionary))


## Moving to Dictionary

In [1]:
import sys
import string
from nltk.stem import PorterStemmer
 
ps = PorterStemmer()

def readFile(file):
#     f = open(f"./ShortStories/{file}.txt", encoding="utf8")
    f=open('./ShortStories/{}.txt'.format(file), encoding='utf-8')
    
    #   Read and Lower Case 
    line = f.read().lower()    
        
    #   Removing Punctuations
    line = line.translate(str.maketrans('','',string.punctuation))

    # We have identified all unique characters and numbers and removed all others but alphabets       
    # '9', '0', '—', 'i', 'u', 'q', '2', 'e', 'ª', 'z', 'y', '”', '8', 'd', 'v', 'o', 'm', 'f', 'b', '¨', '7', '1', 'a', '’', 'l', 'w', '§', 'j', '“', 'x', 'g', '©', 'p', '6', '3', 'r', 'c', 't', 'ã', '5', '´', '¯', '‘', 'h', '4', 's', 'k', 'n
    line = line.translate(str.maketrans('','','1234567890©§“”‘’"´¨¯—ªã'))
        
    for word in line.split():
        word = ps.stem(word)
        if word in dictionary.keys():
            dictionary[word].add(file)
        else:
            dictionary[word] = {file}


    f.close()
    return

dictionary = {}

# Run for Each File
for x in range(1,51):
    readFile(x)
    
    
    
print("Dictionary Length: ", len(dictionary))
# print(dictionary)

Dictionary Length:  6789


In [144]:
print({range(234,237)})

{range(234, 237)}
